<a href="https://colab.research.google.com/github/Aithu-Snehith/Neural-networks-Model-optimisation/blob/master/V1_vgg_pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! nvidia-smi

Thu Jul 11 06:44:49 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    32W /  70W |   1105MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
! pip uninstall -y tensorflow
! pip uninstall -y tf-nightly
! pip install -U tf-nightly-gpu

! pip install tensorflow-model-optimization

Requirement already up-to-date: tf-nightly-gpu in /usr/local/lib/python3.6/dist-packages (1.15.0.dev20190710)


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%load_ext tensorboard
import tensorboard

import tensorflow as tf
tf.enable_eager_execution()

import tempfile
import zipfile
import os

W0711 06:45:09.491101 140456859776896 module_wrapper.py:126] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/util/module_wrapper.py:153: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.



In [0]:
import sys, os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, GaussianDropout, GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import plot_model
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
from tensorflow.keras.layers import *
import time

Using TensorFlow backend.
W0711 06:45:09.954580 140456859776896 module_wrapper.py:126] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/util/module_wrapper.py:153: The name tf.keras.layers.CuDNNGRU is deprecated. Please use tf.compat.v1.keras.layers.CuDNNGRU instead.



In [0]:
path = 'path_to_data'
model_name = 'vgg'

batch_size = 128
num_classes = 7
epochs = 200

# input image dimensions
img_rows, img_cols = 48, 48

In [0]:
# x = np.load(path + 'snehith_fer2013_fdataX.npy')
# y = np.load(path + 'snehith_fer2013_flabels.npy')

# print(x.shape, y.shape)

# x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state = 42)

# np.save(path + 'x_train.npy', x_train)
# np.save(path + 'x_test.npy', x_test)
# np.save(path + 'y_train.npy', y_train)
# np.save(path + 'y_test.npy', y_test)

In [0]:
x_train = np.load(path + 'x_train.npy')
x_test = np.load(path + 'x_test.npy')
y_train = np.load(path + 'y_train.npy')
y_test = np.load(path + 'y_test.npy')

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(32298, 48, 48, 1) (3589, 48, 48, 1) (32298, 7) (3589, 7)


In [0]:
model = Sequential()
# CONV BLOCK 1
model.add(Conv2D(16, (3, 3), input_shape=(img_rows, img_cols, 1), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(GaussianDropout(0.2))
# CONV BLOCK 2
model.add(Conv2D(32, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(GaussianDropout(0.2))
# CONV BLOCK 3
model.add(Conv2D(64, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(GaussianDropout(0.2))
# CONV BLOCK 4
model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(GaussianDropout(0.2))
# CONV BLOCK 5
model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(GaussianDropout(0.2))
# flatten
model.add(Flatten())
# dense 1
model.add(Dense(1024, use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
# dense 2
model.add(Dense(1024, use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 16)        144       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 16)        64        
_________________________________________________________________
activation (Activation)      (None, 48, 48, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 16)        2304      
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 16)        0

In [0]:
logdir = path + model_name + '/logs'
callbacks = [tf.keras.callbacks.TensorBoard(log_dir=logdir, profile_batch=0)]

model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
    metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0711 06:45:12.303889 140456859776896 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1251: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0711 06:45:14.811906 140456859776896 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/optimizer_v2/optimizer_v2.py:460: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


Train on 32298 samples, validate on 3589 samples
Epoch 1/200
32298/32298 [==============================] - 12s 379us/sample - loss: 2.0131 - acc: 0.2551 - val_loss: 1.7559 - val_acc: 0.3070
Epoch 2/200
32298/32298 [==============================] - 6s 191us/sample - loss: 1.6766 - acc: 0.3811 - val_loss: 1.5775 - val_acc: 0.4087
Epoch 3/200
32298/32298 [==============================] - 6s 191us/sample - loss: 1.5275 - acc: 0.4359 - val_loss: 1.4789 - val_acc: 0.4486
Epoch 4/200
32298/32298 [==============================] - 6s 192us/sample - loss: 1.3938 - acc: 0.4723 - val_loss: 1.5005 - val_acc: 0.4193
Epoch 5/200
32298/32298 [==============================] - 6s 193us/sample - loss: 1.2897 - acc: 0.5044 - val_loss: 1.3017 - val_acc: 0.5121
Epoch 6/200
32298/32298 [==============================] - 6s 192us/sample - loss: 1.2322 - acc: 0.5350 - val_loss: 1.3785 - val_acc: 0.4583
Epoch 7/200
32298/32298 [==============================] - 6s 190us/sample - loss: 1.1908 - acc: 0.5479 

In [0]:
# Backend agnostic way to save/restore models
keras_file = path + model_name + '/raw_'+ model_name + '_model.h5'
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Saving model to:  drive/My Drive/interns/Kpit/vgg/raw_vgg_model.h5


In [0]:
from tensorflow_model_optimization.sparsity import keras as sparsity

num_train_samples = x_train.shape[0]
end_step = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * epochs
print('End step: ' + str(end_step))

End step: 50600


In [0]:
pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=2000,
                                                   end_step=end_step,
                                                   frequency=100)
}


pr_model = Sequential()
# CONV BLOCK 1
pr_model.add(Conv2D(16, (3, 3), input_shape=(img_rows, img_cols, 1), padding='same', use_bias=False))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(Conv2D(16, (3, 3), padding='same', use_bias=False))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
pr_model.add(GaussianDropout(0.2))
# CONV BLOCK 2
pr_model.add(Conv2D(32, (3, 3), padding='same', use_bias=False))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(Conv2D(32, (3, 3), padding='same', use_bias=False))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
pr_model.add(GaussianDropout(0.2))
# CONV BLOCK 3
pr_model.add(Conv2D(64, (3, 3), padding='same', use_bias=False))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(Conv2D(64, (3, 3), padding='same', use_bias=False))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
pr_model.add(GaussianDropout(0.2))
# CONV BLOCK 4
pr_model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
pr_model.add(GaussianDropout(0.2))
# CONV BLOCK 5
pr_model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
pr_model.add(GaussianDropout(0.2))
# flatten
pr_model.add(Flatten())
# dense 1
pr_model.add(Dense(1024, use_bias=False))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(Dropout(0.5))
# dense 2
pr_model.add(Dense(1024, use_bias=False))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
pr_model.add(Dropout(0.5))
pr_model.add(Dense(num_classes, activation='softmax'))
pr_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 48, 48, 16)        144       
_________________________________________________________________
batch_normalization_12 (Batc (None, 48, 48, 16)        64        
_________________________________________________________________
activation_12 (Activation)   (None, 48, 48, 16)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 48, 48, 16)        2304      
_________________________________________________________________
batch_normalization_13 (Batc (None, 48, 48, 16)        64        
_________________________________________________________________
activation_13 (Activation)   (None, 48, 48, 16)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 24, 24, 16)       

In [0]:
pr_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

# Add a pruning step callback to peg the pruning step to the optimizer's
# step. Also add a callback to add pruning summaries to tensorboard
callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

pr_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks,
          validation_data=(x_test, y_test))

score = pr_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 32298 samples, validate on 3589 samples
Epoch 1/200
32298/32298 [==============================] - 10s 306us/sample - loss: 1.9946 - acc: 0.2389 - val_loss: 1.7203 - val_acc: 0.3001
Epoch 2/200
32298/32298 [==============================] - 6s 194us/sample - loss: 1.6976 - acc: 0.3694 - val_loss: 1.8131 - val_acc: 0.3912
Epoch 3/200
32298/32298 [==============================] - 6s 195us/sample - loss: 1.5400 - acc: 0.4341 - val_loss: 1.6548 - val_acc: 0.3912
Epoch 4/200
32298/32298 [==============================] - 6s 197us/sample - loss: 1.4026 - acc: 0.4690 - val_loss: 1.3029 - val_acc: 0.5088
Epoch 5/200
32298/32298 [==============================] - 6s 198us/sample - loss: 1.2955 - acc: 0.5075 - val_loss: 1.2979 - val_acc: 0.4990
Epoch 6/200
32298/32298 [==============================] - 6s 195us/sample - loss: 1.2334 - acc: 0.5308 - val_loss: 1.3174 - val_acc: 0.5191
Epoch 7/200
32298/32298 [==============================] - 6s 192us/sample - loss: 1.1871 - acc: 0.5527 

In [0]:
checkpoint_file = path + model_name + '/pruned_'+ model_name + '_checkpoint.h5'
print('Saving pruned model to: ', checkpoint_file)
# saved_model() sets include_optimizer to True by default. Spelling it out here
# to highlight.
tf.keras.models.save_model(pr_model, checkpoint_file, include_optimizer=True)

with sparsity.prune_scope():
  restored_model = tf.keras.models.load_model(checkpoint_file)

restored_model.fit(x_train, y_train,
                   batch_size=batch_size,
                   epochs=int(epochs/10),
                   verbose=1,
                   callbacks=callbacks,
                   validation_data=(x_test, y_test))

score = restored_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Saving pruned model to:  drive/My Drive/interns/Kpit/vgg/pruned_vgg_checkpoint.h5
Train on 32298 samples, validate on 3589 samples
Epoch 1/20
32298/32298 [==============================] - 9s 291us/sample - loss: 0.1399 - acc: 0.9504 - val_loss: 1.7989 - val_acc: 0.6606
Epoch 2/20
32298/32298 [==============================] - 6s 191us/sample - loss: 0.1386 - acc: 0.9504 - val_loss: 1.7758 - val_acc: 0.6562
Epoch 3/20
32298/32298 [==============================] - 6s 192us/sample - loss: 0.1425 - acc: 0.9495 - val_loss: 1.7861 - val_acc: 0.6525
Epoch 4/20
32298/32298 [==============================] - 6s 192us/sample - loss: 0.1346 - acc: 0.9535 - val_loss: 1.8560 - val_acc: 0.6606
Epoch 5/20
32298/32298 [==============================] - 6s 193us/sample - loss: 0.1370 - acc: 0.9512 - val_loss: 1.8033 - val_acc: 0.6601
Epoch 6/20
32298/32298 [==============================] - 6s 193us/sample - loss: 0.1430 - acc: 0.9505 - val_loss: 1.8162 - val_acc: 0.6481
Epoch 7/20
32298/32298 [=====

In [0]:
final_model = sparsity.strip_pruning(pr_model)
final_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 48, 48, 16)        144       
_________________________________________________________________
batch_normalization_12 (Batc (None, 48, 48, 16)        64        
_________________________________________________________________
activation_12 (Activation)   (None, 48, 48, 16)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 48, 48, 16)        2304      
_________________________________________________________________
batch_normalization_13 (Batc (None, 48, 48, 16)        64        
_________________________________________________________________
activation_13 (Activation)   (None, 48, 48, 16)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 24, 24, 16)       

In [0]:
pruned_keras_file = path + model_name + '/pruned_'+ model_name + '.h5'
print('Saving pruned model to: ', pruned_keras_file)

# No need to save the optimizer with the graph for serving.
tf.keras.models.save_model(final_model, pruned_keras_file, include_optimizer=False)

Saving pruned model to:  drive/My Drive/interns/Kpit/vgg/pruned_vgg.h5


In [0]:
zip1 = path + model_name + '/raw_'+ model_name + '.zip' 
with zipfile.ZipFile(zip1, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(keras_file)
print("Size of the unpruned model before compression: %.2f Mb" % 
      (os.path.getsize(keras_file) / float(2**20)))
print("Size of the unpruned model after compression: %.2f Mb" % 
      (os.path.getsize(zip1) / float(2**20)))

zip2 = path + model_name + '/pruned_'+ model_name + '.zip' 
with zipfile.ZipFile(zip2, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(pruned_keras_file)
print("Size of the pruned model before compression: %.2f Mb" % 
      (os.path.getsize(pruned_keras_file) / float(2**20)))
print("Size of the pruned model after compression: %.2f Mb" % 
      (os.path.getsize(zip2) / float(2**20)))

Size of the unpruned model before compression: 8.43 Mb
Size of the unpruned model after compression: 7.72 Mb
Size of the pruned model before compression: 8.43 Mb
Size of the pruned model after compression: 7.72 Mb


In [0]:
tflite_model_file = path + model_name + '/pruned_'+ model_name + '.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model_file(pruned_keras_file)
tflite_model = converter.convert()
with open(tflite_model_file, 'wb') as f:
  f.write(tflite_model)

W0711 07:28:53.874749 140456859776896 hdf5_format.py:171] No training configuration found in save file: the model was *not* compiled. Compile it manually.
W0711 07:28:54.569505 140456859776896 module_wrapper.py:126] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/util/module_wrapper.py:153: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.



In [0]:
zip_tflite = path + model_name + '/pruned_'+ model_name + '_tflite.zip'
with zipfile.ZipFile(zip_tflite, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(tflite_model_file)
print("Size of the tflite model before compression: %.2f Mb" 
      % (os.path.getsize(tflite_model_file) / float(2**20)))
print("Size of the tflite model after compression: %.2f Mb" 
      % (os.path.getsize(zip_tflite) / float(2**20)))

Size of the tflite model before compression: 8.29 Mb
Size of the tflite model after compression: 7.69 Mb


In [0]:
import numpy as np

interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

def eval_model(interpreter, x_test, y_test):
  total_seen = 0
  num_correct = 0

  for img, label in zip(x_test, y_test):
    inp = img.reshape((1, 48, 48, 1))
    total_seen += 1
    interpreter.set_tensor(input_index, inp)
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_index)
    if np.argmax(predictions) == np.argmax(label):
      num_correct += 1

    if total_seen % 1000 == 0:
        print("Accuracy after %i images: %f" %
              (total_seen, float(num_correct) / float(total_seen)))

  return float(num_correct) / float(total_seen)


print(eval_model(interpreter, x_test, y_test))

Accuracy after 1000 images: 0.670000
Accuracy after 2000 images: 0.672500
Accuracy after 3000 images: 0.660333
0.6617442184452493


In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(pruned_keras_file)

converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

tflite_quant_model = converter.convert()

tflite_quant_model_file = path + model_name + '/pruned_'+ model_name + '_quant.tflite'
with open(tflite_quant_model_file, 'wb') as f:
  f.write(tflite_quant_model)

W0711 07:29:14.805264 140456859776896 hdf5_format.py:171] No training configuration found in save file: the model was *not* compiled. Compile it manually.


In [0]:
zip_tflite = path + model_name + '/pruned_quant_'+ model_name + '_tflite.zip'
with zipfile.ZipFile(zip_tflite, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(tflite_quant_model_file)
print("Size of the tflite model before compression: %.2f Mb" 
      % (os.path.getsize(tflite_quant_model_file) / float(2**20)))
print("Size of the tflite model after compression: %.2f Mb" 
      % (os.path.getsize(zip_tflite) / float(2**20)))

Size of the tflite model before compression: 2.08 Mb
Size of the tflite model after compression: 1.59 Mb


In [0]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_quant_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

num_correct = 0
aa = time.time()
for img,label in zip(x_test, y_test):
    inp = img.reshape((1,48,48,1))
    interpreter.set_tensor(input_index, inp)
    interpreter.invoke()
    output = interpreter.get_tensor(output_index)
    if np.argmax(output) == np.argmax(label):
        num_correct = num_correct + 1
time_elapsed = time.time() - aa

print('total_images : ', len(x_test), 'accuracy : ' , float(num_correct)/len(x_test))
print('total time :', time_elapsed , 'time per image: ', time_elapsed/len(x_test), 'fps : ', len(x_test)/time_elapsed)


total_images :  3589 accuracy :  0.6628587350236835
total time : 24.232789516448975 time per image:  0.006751961414446636 fps :  148.10511177690967
